<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/news_decoder_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tbody><tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a rel="noopener" href="https://www.thelmbook.com">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a rel="noopener" href="https://github.com/aburkov/theLMbook">https://github.com/aburkov/theLMbook</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </tbody></table>
    </div>
</div>

# 디코더 기반 언어 모델

## 유틸리티 함수와 클래스

다음 셀에서 필요 라이브러리를 임포트하고 유틸리티 함수와 모델 클래스를 정의합니다.

In [ ]:
# 필수 라이브러리 임포트
import os               # 파일 및 경로 작업을 위한 라이브러리 (check_file_exists, extract_dataset)
import urllib.request   # URL에서 데이터셋 파일 다운로드를 위한 라이브러리
import tarfile          # .tar.gz 데이터셋 압축 해제를 위한 라이브러리
import torch            # 텐서 연산과 딥러닝을 위한 메인 파이토치 라이브러리
import torch.nn as nn   # 신경망 모듈, 층 및 유틸리티
import torch.nn.functional as F  # 소프트맥스를 위한 라이브러리
from torch.utils.data import DataLoader, IterableDataset  # 효율적인 데이터 로딩을 위한 라이브러리
import random           # 난수 시드 설정을 위한 라이브러리
from tqdm import tqdm   # 진행률 표시줄을 위한 라이브러리
import math             # exp()를 사용한 혼잡도 계산을 위한 라이브러리
import re               # 텍스트 전처리를 위한 라이브러리 (숫자를 자리 표시자로 대체)
from transformers import AutoTokenizer  # 사전 훈련된 토크나이저 로드를 위한 라이브러리
#import tempfile         # 추출 중 임시 파일 처리를 위한 라이브러리
#import shutil           # 추출 중 파일 작업을 위한 라이브러리

# ----------------------------
# 유틸리티 함수
# ----------------------------
def set_seed(seed):
    """
    다양한 파이썬 라이브러리에서 재현성을 위해 난수 시드를 설정합니다.
    이렇게 하면 실행 시간에 걸쳐 무작위 연산이 동일한 결과를 제공합니다.
    Args:
        seed (int): 난수 생성을 위한 시드 값
    """
    # 파이썬 내장 random 모듈의 시드 설정
    random.seed(seed)
    # 파이토치의 CPU 난수 생성기 시드 설정
    torch.manual_seed(seed)
    # 파이토치의 GPU 난수 생성기 시드 설정
    torch.cuda.manual_seed_all(seed)
    # 가능한 경우 cuDNN이 결정론적 알고리즘을 사용하도록 요청
    # 참고: 이는 성능에 영향을 미칠 수 있으며 모든 경우에서 결정론을 보장하지 않을 수 있습니다
    torch.backends.cudnn.deterministic = True
    # 특정 입력 크기에 대해 최적의 알고리즘을 찾는 cuDNN의 자동 튜너를 비활성화
    # 일관된 동작을 보장하지만 입력 크기를 최적화하지 않으므로 더 느릴 수 있습니다
    torch.backends.cudnn.benchmark = False

# ----------------------------
# 데이터셋 클래스
# ----------------------------
class IterableTextDataset(IterableDataset):
    """
    메모리 효율적인 방식으로 텍스트 데이터를 처리하기 위한 반복 가능한 데이터셋입니다.
    모든 데이터를 메모리에 로드하는 대신 디스크에서 데이터를 스트리밍합니다.
    스트리밍 지원을 위해 파이토치의 IterableDataset을 상속합니다.
    Args:
        file_path (str): 문장이 포함된 텍스트 파일의 경로
        tokenizer: 텍스트를 토큰으로 변환하기 위한 토크나이저 객체
        max_length (int): 처리할 최대 시퀀스 길이 (기본값: 30)
    """
    def __init__(self, file_path, tokenizer, max_length=30):
        # 데이터 읽기를 위한 파일 경로 저장
        self.file_path = file_path
        # 텍스트 처리를 위한 토크나이저 저장
        self.tokenizer = tokenizer
        # 긴 시퀀스를 자르기 위한 최대 시퀀스 길이 설정
        self.max_length = max_length
        self._count_sentences()

    def __iter__(self):
        """
        데이터셋에 대한 반복자를 생성합니다.
        이 메서드는 데이터셋을 반복할 때 호출됩니다.
        Yields:
            tuple: 언어 모델링을 위한 (input_sequence, target_sequence) 쌍
                  input_sequence는 마지막 토큰까지의 시퀀스
                  target_sequence는 한 위치 오른쪽으로 이동한 시퀀스
        """
        # UTF-8 인코딩으로 읽기 모드에서 파일 열기
        with open(self.file_path, 'r', encoding="utf-8") as f:
            # 파일의 각 라인(문장) 처리
            for line in f:
                # 앞뒤 공백 제거
                sentence = line.strip()
                # 모든 숫자를 ### 자리 표시자로 대체
                # 이는 어휘 크기를 줄이고 모델의 일반화에 도움이 됩니다
                sentence = re.sub(r"\d+", "###", sentence)
                # 문장을 토큰 ID로 변환
                encoded_sentence = self.tokenizer.encode(
                    sentence,
                    max_length=self.max_length,
                    truncation=True
                )
                # 최소 2개의 토큰이 있는 시퀀스만 사용
                # (최소 하나의 입력과 하나의 타깃 토큰이 필요)
                if len(encoded_sentence) >= 2:
                    # 입력은 마지막을 제외한 모든 토큰
                    input_seq = encoded_sentence[:-1]
                    # 타깃은 첫 번째를 제외한 모든 토큰
                    target_seq = encoded_sentence[1:]
                    # 파이토치 텐서로 변환하여 반환
                    yield torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_seq, dtype=torch.long)

    def __len__(self):
        return self._num_sentences

    def _count_sentences(self):
        print(f"\n{self.file_path}에서 문장 수 계산 중...")
        with open(self.file_path, 'r', encoding="utf-8") as f:
            self._num_sentences = sum(1 for _ in f)
        print(f"\n{self.file_path}에서 {self._num_sentences}개의 문장을 찾았습니다.")

## ----------------------------
## 데이터 다운로드 및 준비
## ----------------------------
def create_collate_fn(tokenizer):
    """
    다양한 길이의 시퀀스를 배치로 만들기 위한 collate 함수를 생성합니다.
    이 함수는 더 짧은 시퀀스를 배치에서 가장 긴 시퀀스와 일치하도록 패딩합니다.
    Args:
        tokenizer: 패딩 토큰 정보를 포함하는 토크나이저 객체
    Returns:
        function: 배치에서 패딩을 처리하는 collate 함수
    """
    def collate_fn(batch):
        # 배치에서 입력과 타깃 분리
        input_seqs, target_seqs = zip(*batch)
        # 토크나이저에서 패딩 토큰 ID 가져오기
        pad_index = tokenizer.pad_token_id
        # 입력 시퀀스를 동일한 길이로 패딩
        input_padded = nn.utils.rnn.pad_sequence(input_seqs, batch_first=True, padding_value=pad_index)
        # 타깃 시퀀스를 동일한 길이로 패딩
        target_padded = nn.utils.rnn.pad_sequence(target_seqs, batch_first=True, padding_value=pad_index)
        return input_padded, target_padded
    return collate_fn

def check_file_exists(filename):
    """
    현재 디렉토리에 파일이 있는지 확인합니다.
    Args:
        filename (str): 확인할 파일 이름
    Returns:
        bool: 파일이 있으면 True, 없으면 False
    """
    return os.path.exists(filename)

def download_file(url):
    """
    로컬에 파일이 없는 경우 주어진 URL에서 파일을 다운로드합니다.
    다운로드 차단을 방지하기 위해 사용자 지정 User-Agent를 사용합니다.
    Args:
        url (str): 다운로드할 파일의 URL
    Returns:
        str: 다운로드된 파일의 이름 ("news.tar.gz")
    """
    # URL에 관계없이 항상 news.tar.gz를 파일 이름으로 사용
    filename = "news.tar.gz"
    if not check_file_exists(filename):
        print(f"\n{url}에서 데이터셋 다운로드 중...")
        req = urllib.request.Request(
            url,
            headers={"User-Agent": "Mozilla/5.0"}
        )
        with urllib.request.urlopen(req) as response:
            with open(filename, "wb") as out_file:
                out_file.write(response.read())
        print("\n다운로드가 완료되었습니다.")
    else:
        print(f"\n{filename}은(는) 이미 다운로드되었습니다.")
    return filename

def is_within_directory(directory, target):
    """
    절대 경로를 비교하여 대상 경로가 지정된 디렉토리 내에 있는지 확인합니다.
    Args:
        directory (str): 기본 디렉토리 경로
        target (str): 확인할 대상 경로
    Returns:
        bool: 대상의 절대 경로가 디렉토리의 절대 경로로 시작하면 True
    """
    abs_directory = os.path.abspath(directory)
    abs_target = os.path.abspath(target)
    prefix = os.path.commonprefix([abs_directory, abs_target])
    return prefix == abs_directory

def extract_dataset(filename):
    """
    다운로드한 아카이브에서 train.txt와 test.txt를 추출합니다.
    아카이브 내용에 대한 디버그 정보를 포함합니다.
    Args:
        filename (str): 아카이브 파일의 이름
    Returns:
        tuple: 추출된 train과 test 파일의 경로
    """
    data_dir = os.path.join(os.path.dirname(filename), "news")
    train_path = os.path.join(data_dir, "train.txt")
    test_path = os.path.join(data_dir, "test.txt")

    if check_file_exists(train_path) and check_file_exists(test_path):
        print("\n데이터 파일이 이미 추출되었습니다.")
        return train_path, test_path

    print("\n아카이브 내용 나열:")
    with tarfile.open(filename, "r:gz") as tar:
        for member in tar.getmembers():
            print(f"\n아카이브 멤버: {member.name}")
        print("\n파일 추출 중...")
        # 먼저 현재 디렉토리로 추출
        tar.extractall('.')

    if not (check_file_exists(train_path) and check_file_exists(test_path)):
        raise FileNotFoundError(f"\n아카이브에서 필요한 파일을 찾을 수 없습니다. 위의 경로를 확인하십시오.")

    print("\n추출이 완료되었습니다.")
    return train_path, test_path

def create_datasets(train_file, test_file, tokenizer, max_length=30):
    """
    훈련과 테스트를 위한 IterableTextDataset 객체를 생성합니다.
    이 데이터셋은 모든 데이터를 메모리에 로드하는 대신 디스크에서 데이터를 스트리밍합니다.
    Args:
        train_file (str): 훈련 데이터 파일의 경로
        test_file (str): 테스트 데이터 파일의 경로
        tokenizer: 텍스트 처리를 위한 토크나이저 객체
    Returns:
        tuple: (train_dataset, test_dataset) - 훈련과 테스트를 위한 데이터셋 객체
    """
    # 훈련 데이터셋 생성
    train_dataset = IterableTextDataset(train_file, tokenizer, max_length)
    # 테스트 데이터셋 생성
    test_dataset = IterableTextDataset(test_file, tokenizer, max_length)
    # 데이터셋 크기 출력
    print(f"\n훈련 문장 수: {len(train_dataset)}")
    print(f"\n테스트 문장 수: {len(test_dataset)}")
    return train_dataset, test_dataset

def create_dataloaders(train_dataset, test_dataset, batch_size, collate_fn):
    """
    효율적인 데이터 반복을 위한 DataLoader 객체를 생성합니다.
    Args:
        train_dataset: 훈련 데이터셋
        test_dataset: 테스트 데이터셋
        batch_size (int): 배치당 시퀀스 수
        collate_fn: 패딩과 배치 생성을 처리하는 함수
    Returns:
        tuple: (train_dataloader, test_dataloader) - 적절한 패딩으로
               데이터 배치를 반복하기 위한 DataLoader 객체
    """
    # 훈련 데이터 로더 생성
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,    # 패딩을 처리하는 함수
        num_workers=0             # 워커 프로세스 수 (0 = 단일 프로세스)
    )
    # 테스트 데이터 로더 생성
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        num_workers=0
    )
    return train_dataloader, test_dataloader

def download_and_prepare_data(url, batch_size, tokenizer, max_length=30):
    """
    전체 데이터 준비 파이프라인을 처리하는 메인 함수입니다.
    데이터를 다운로드하고, 추출하고, 필요한 데이터셋 객체를 생성합니다.
    Args:
        url (str): 데이터셋 아카이브를 다운로드할 수 있는 URL
        batch_size (int): 데이터 로딩을 위한 배치 크기
        tokenizer: 텍스트 처리를 위한 토크나이저 객체
        max_length (int): 토큰화를 위한 최대 시퀀스 길이 (기본값: 30)
    Returns:
        tuple: (train_dataloader, test_dataloader) - 사용 준비된 데이터 로더
    """
    # 1단계: URL에서 데이터셋 아카이브 다운로드
    filename = download_file(url)
    # 2단계: 아카이브에서 훈련 및 테스트 파일 추출
    train_file, test_file = extract_dataset(filename)
    # 3단계: 데이터 스트리밍을 위한 데이터셋 객체 생성
    train_dataset, test_dataset = create_datasets(train_file, test_file, tokenizer, max_length)
    # 4단계: 배치 생성을 처리하는 함수 생성
    collate_fn = create_collate_fn(tokenizer)
    # 5단계: 데이터 로더 생성 및 반환
    return create_dataloaders(train_dataset, test_dataset, batch_size, collate_fn)

# ----------------------------
# 평가 함수
# ----------------------------
def compute_loss_and_perplexity(model, dataloader, tokenizer, criterion, device, max_sentences=1000):
    """
    데이터에 대한 손실과 혼잡도를 계산하여 모델 성능을 평가합니다.
    Args:
        model (nn.Module): 평가할 언어 모델
        dataloader (DataLoader): 배치 시퀀스를 포함하는 데이터 로더
        tokenizer: 패딩과 같은 특수 토큰을 처리하기 위한 토크나이저
        criterion: 손실 함수 (보통 CrossEntropyLoss)
        device: 계산을 실행할 장치 (cuda/cpu)
        max_sentences (int): 평가할 최대 문장 수 (기본값: 1000)
                           더 빠른 검증을 위해 평가를 하위 집합으로 제한합니다
    Returns:
        tuple: (average_loss, perplexity)
               - average_loss: 토큰당 평균 손실 (패딩 제외)
               - perplexity: exp(average_loss), 낮을수록 좋음
    """
    # 모델을 평가 모드로 설정 (드롭아웃 등 비활성화)
    model.eval()
    # 손실 계산을 위한 카운터 초기화
    total_loss = 0.0          # 모든 배치에 대한 총 손실 누적
    total_tokens = 0          # 총 토큰 수 카운터 (패딩 제외)
    sentences_processed = 0    # 처리된 문장 수 카운터

    # 효율성을 위해 그레이디언트 계산 비활성화
    with torch.no_grad():
        # 진행률 표시와 함께 데이터 반복
        for input_seq, target_seq in tqdm(dataloader, desc="평가 중", leave=False):
            # 입력과 타깃 시퀀스를 지정된 장치로 이동
            input_seq = input_seq.to(device)      # 크기: (batch_size, seq_len)
            target_seq = target_seq.to(device)    # 크기: (batch_size, seq_len)

            # 현재 배치 크기 가져오기 (마지막 배치는 더 작을 수 있음)
            batch_size_current = input_seq.size(0)

            # 모델을 통한 포워드 패스
            logits = model(input_seq)             # 크기: (batch_size, seq_len, vocab_size)

            # 손실 계산을 위해 로짓과 타깃 재구성
            logits = logits.reshape(-1, logits.size(-1))  # 크기: (batch_size * seq_len, vocab_size)
            target = target_seq.reshape(-1)              # 크기: (batch_size * seq_len)

            # 패딩 토큰을 제외하는 마스크 생성
            mask = target != tokenizer.pad_token_id

            # 패딩되지 않은 토큰에 대해서만 손실 계산
            loss = criterion(logits[mask], target[mask])

            # 카운터 업데이트
            loss_value = loss.item() * mask.sum().item()  # 이 배치의 총 손실
            total_loss += loss_value                      # 배치 손실 누적
            total_tokens += mask.sum().item()             # 패딩이 아닌 토큰 수 계산

            # 문장 카운터 업데이트 및 최대값에 도달했는지 확인
            sentences_processed += batch_size_current
            if sentences_processed >= max_sentences:
                break

    # 최종 메트릭 계산
    average_loss = total_loss / total_tokens           # 토큰 수로 손실 정규화
    perplexity = math.exp(average_loss)               # 손실을 혼잡도로 변환
    return average_loss, perplexity

def generate_text(model, start_string, tokenizer, device, max_length=50):
    """
    탐욕적 디코딩을 사용하여 주어진 시작 문자열에서 텍스트 연속을 생성합니다.
    Args:
        model (nn.Module): 훈련된 언어 모델
        start_string (str): 계속할 초기 텍스트
        tokenizer: 텍스트 처리를 위한 토크나이저
        device: 생성을 실행할 장치 (cuda/cpu)
        max_length (int): 생성된 시퀀스의 최대 길이
    Returns:
        str: 생성된 텍스트 연속
    """
    # 드롭아웃 및 기타 훈련 특정 동작을 비활성화하도록 모델을 평가 모드로 설정
    model.eval()

    # 입력 문자열을 토큰 인덱스로 변환
    input_indices = tokenizer.encode(start_string, add_special_tokens=False)

    # 인덱스를 텐서로 변환하고 지정된 장치(GPU/CPU)로 이동
    input_tensor = torch.tensor([input_indices], dtype=torch.long).to(device)

    # 입력 시퀀스로 시작하여 생성된 모든 토큰 추적
    generated_indices = input_indices.copy()

    # 최대 길이에 도달하거나 시퀀스 끝 토큰을 만날 때까지 토큰 생성
    for _ in range(max_length - len(input_indices)):
        # 전체 시퀀스에 대한 모델 예측 가져오기
        logits = model(input_tensor)

        # 마지막 토큰 위치에 대한 예측만 가져오기
        logits = logits[:, -1, :]

        # 알 수 없는 토큰의 확률을 음의 무한대로 설정하여 모델이 알 수 없는 토큰을 생성하지 않도록 방지
        if tokenizer.unk_token_id is not None:
            logits[:, tokenizer.unk_token_id] = float("-inf")

        # 탐욕적 디코딩: 가장 높은 확률을 가진 토큰 선택
        next_token = torch.argmax(logits, dim=-1)

        # 선택한 토큰을 생성된 시퀀스에 추가
        generated_indices.append(next_token.item())

        # 시퀀스 끝 토큰을 생성하면 생성 중지
        if next_token.item() == tokenizer.eos_token_id:
            break

        # 다음 반복을 위해 입력 텐서에 새 토큰 추가
        input_tensor = torch.cat([input_tensor, next_token.unsqueeze(0)], dim=1)

    # 특수 토큰을 제거하고 토큰 인덱스를 다시 텍스트로 변환
    return tokenizer.decode(generated_indices, skip_special_tokens=True)

def save_model(model, tokenizer, model_name):
    """
    지정된 모델 이름을 사용하여 모델 상태 사전과 토크나이저를 저장합니다.
    Args:
        model (nn.Module): 저장할 훈련된 모델
        tokenizer: 모델과 함께 사용된 토크나이저
        model_name (str): 저장된 모델 파일에 사용할 이름
    """
    # 모델 디렉토리가 없는 경우 생성
    save_dir = os.path.join("models", model_name)
    os.makedirs(save_dir, exist_ok=True)

    # 모델 상태 딕셔너리와 설정 저장
    model_path = os.path.join(save_dir, f"{model_name}.pth")
    torch.save({
        "model_state_dict": model.state_dict(),
        "model_config": {
            "vocab_size": len(tokenizer),
            "emb_dim": model.embedding.embedding_dim,
            "num_heads": len(model.layers[0].attn.heads),
            "num_blocks": len(model.layers),
            "pad_idx": model.embedding.padding_idx
        }
    }, model_path)

    # 토크나이저 저장
    tokenizer_path = os.path.join(save_dir, "tokenizer")
    tokenizer.save_pretrained(tokenizer_path)

    print(f"모델과 토크나이저가 '{model_name}'(으)로 저장되었습니다")

def load_model(model_name, device=None):
    """
    모델 이름을 사용하여 저장된 모델과 토크나이저를 로드합니다.
    Args:
        model_name (str): 로드할 모델의 이름
        device: 모델을 로드할 장치 (None인 경우 사용 가능한 장치 사용)
    Returns:
        tuple: (loaded_model, loaded_tokenizer)
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    save_dir = os.path.join("models", model_name)

    # 모델이 존재하는지 확인
    if not os.path.exists(save_dir):
        raise FileNotFoundError(f"'{model_name}' 이름으로 저장된 모델을 찾을 수 없습니다")

    # 토크나이저 로드
    tokenizer_path = os.path.join(save_dir, "tokenizer")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    # 모델 상태와 설정 로드
    model_path = os.path.join(save_dir, f"{model_name}.pth")
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)

    # 저장된 설정으로 새 모델 인스턴스 생성
    model = DecoderLanguageModel(
        vocab_size=checkpoint["model_config"]["vocab_size"],
        emb_dim=checkpoint["model_config"]["emb_dim"],
        num_heads=checkpoint["model_config"]["num_heads"],
        num_blocks=checkpoint["model_config"]["num_blocks"],
        pad_idx=checkpoint["model_config"]["pad_idx"]
    )

    # 저장된 상태 딕셔너리 로드
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()

    print(f"\n'{model_name}' 모델이 성공적으로 로드되었습니다")
    return model, tokenizer

def get_hyperparameters():
    emb_dim = 128
    num_heads = 8
    num_blocks = 2
    batch_size = 128
    learning_rate = 0.001
    num_epochs = 1
    context_size = 30
    return emb_dim, num_heads, num_blocks, batch_size, learning_rate, num_epochs, context_size

# 모델 클래스

디코더 트랜스포머 언어 모델 클래스와 초기화 메서드

In [ ]:
# ----------------------------
# 가중치 초기화 및 핵심 함수
# 이 섹션에는 가중치 초기화를 위한 유틸리티 함수와
# 모델 전체에서 사용되는 핵심 계산 함수가 포함되어 있습니다
# ----------------------------
def initialize_weights(model):
    """
    적절한 방식을 사용하여 다양한 모델 구성 요소의 가중치를 초기화합니다.
    각 층 유형은 최적의 훈련을 위해 특화된 초기화를 받습니다.
    """
    for module in model.modules():
        if isinstance(module, nn.Linear):
            # 선형 층에 대한 Xavier 균등 초기화
            # 네트워크 층 전체에 걸쳐 분산을 유지하는 데 도움이 됩니다
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)  # 편향을 0으로 초기화
        elif isinstance(module, nn.Embedding):
            # 정규 분포를 사용하여 임베딩 층 초기화
            nn.init.normal_(module.weight, mean=0, std=0.02)
            if module.padding_idx is not None:
                # 패딩 토큰이 0 임베딩을 갖도록 보장
                with torch.no_grad():
                    module.weight[module.padding_idx].fill_(0)
        elif isinstance(module, AttentionHead):
            # 쿼리, 키, 값 투영 행렬 초기화
            # Xavier 균등은 좋은 그레이디언트 흐름을 유지하는 데 도움이 됩니다
            nn.init.xavier_uniform_(module.W_Q)
            nn.init.xavier_uniform_(module.W_K)
            nn.init.xavier_uniform_(module.W_V)
        elif isinstance(module, MultiHeadAttention):
            # 어텐션 메커니즘을 위한 출력 투영 행렬 초기화
            nn.init.xavier_uniform_(module.W_O)
        elif isinstance(module, DecoderLanguageModel):
            # 최종 출력 투영 층 초기화
            nn.init.xavier_uniform_(module.output)
        elif isinstance(module, RMSNorm):
            # RMSNorm 스케일 매개변수를 1로 초기화
            # 이는 항등 변환으로 시작합니다
            nn.init.ones_(module.scale)
        elif isinstance(module, MLP):
            # 피드-포워드 네트워크 매개변수 초기화
            nn.init.xavier_uniform_(module.W_1)
            nn.init.xavier_uniform_(module.W_2)
            nn.init.zeros_(module.B_1)
            nn.init.zeros_(module.B_2)

def rope(x, theta_base=10000.0):
    """
    트랜스포머 어텐션을 위한 로터리 위치 임베딩(RoPE)을 구현합니다.
    RoPE는 차원 쌍에 적용된 회전 행렬을 통해 위치 정보를 인코딩합니다.
    Args:
        x: 크기가 (batch_size, seq_len, emb_dim)인 입력 텐서
        theta_base: 회전 주파수 계산을 위한 기준 (기본값: 10000.0)
    Returns:
        회전을 통해 위치 정보가 인코딩된 텐서
    """
    batch_size, seq_len, emb_dim = x.size()
    assert emb_dim % 2 == 0, "RoPE를 위해서는 임베딩 차원이 짝수여야 합니다"

    # 시퀀스 위치 인덱스 생성
    pos = torch.arange(0, seq_len, dtype=torch.float32, device=x.device)
    pos = pos.unsqueeze(0).expand(batch_size, seq_len)

    # 각 차원 쌍에 대한 주파수 대역 계산
    # 수정: 주파수는 p=1에서 시작하고 (p-1)을 지수로 사용
    p = torch.arange(1, emb_dim // 2 + 1, dtype=torch.float32, device=x.device)
    theta_p = 1.0 / (theta_base ** (2 * (p - 1) / emb_dim))

    # 각 위치와 주파수에 대한 회전 각도 계산
    pos = pos.unsqueeze(-1)
    theta = pos * theta_p

    # 회전 구성 요소 계산
    sin_theta = torch.sin(theta)
    cos_theta = torch.cos(theta)

    # 입력을 교대 차원으로 분할
    x1 = x[..., 0::2]  # 인덱스 0,2,4,...의 차원
    x2 = x[..., 1::2]  # 인덱스 1,3,5,...의 차원

    # 각 쌍에 2D 회전 적용
    x_rotated_1 = x1 * cos_theta - x2 * sin_theta
    x_rotated_2 = x1 * sin_theta + x2 * cos_theta

    # 회전된 쌍을 최종 출력으로 재결합
    x_rotated = torch.stack((x_rotated_1, x_rotated_2), dim=-1).reshape(batch_size, seq_len, emb_dim)
    return x_rotated

# ----------------------------
# 모델 구성 요소
# 이 섹션에는 정규화, 어텐션, 피드-포워드 층을 포함한
# 트랜스포머 디코더의 빌딩 블록이 포함되어 있습니다
# ----------------------------
class RMSNorm(nn.Module):
    """
    제곱 평균 제곱근 층 정규화
    RMS 통계만 사용하는 층 정규화의 단순화된 대안입니다
    """
    def __init__(self, emb_dim, epsilon=1e-8):
        super().__init__()
        self.scale = nn.Parameter(torch.ones(emb_dim))  # 학습 가능한 스케일 매개변수
        self.epsilon = epsilon  # 수치적 안정성을 위한 작은 상수

    def forward(self, x):
        # 제곱 평균 제곱근 정규화 계산
        squared_x = x ** 2
        mean_squared = torch.mean(squared_x, dim=-1, keepdim=True)
        rms = torch.sqrt(mean_squared + self.epsilon)
        # 정규화 및 스케일링
        x_normalized = x / rms
        output = x_normalized * self.scale
        return output

class AttentionHead(nn.Module):
    """
    단일 어텐션 헤드
    학습된 투영을 사용하여 입력을 변환하고 스케일링된 내적 어텐션을 계산합니다
    """
    def __init__(self, emb_dim, d_h):
        super().__init__()
        # 쿼리, 키, 값에 대한 투영 행렬 초기화
        self.W_Q = nn.Parameter(torch.rand(emb_dim, d_h))
        self.W_K = nn.Parameter(torch.rand(emb_dim, d_h))
        self.W_V = nn.Parameter(torch.rand(emb_dim, d_h))
        self.d_h = d_h  # 어텐션 헤드의 차원

    def forward(self, x, mask):
        # 입력을 쿼리, 키, 값 공간으로 투영
        Q = x @ self.W_Q
        K = x @ self.W_K
        V = x @ self.W_V

        # 쿼리와 키에 로터리 위치 임베딩 적용
        Q, K = rope(Q), rope(K)

        # 스케일링 팩터로 어텐션 점수 계산
        scores = Q @ K.transpose(-2, -1) / math.sqrt(self.d_h)

        # 코잘 마스크와 어텐션 가중치 적용
        masked_scores = scores.masked_fill(mask == 0, float("-inf"))
        attention_weights = torch.softmax(masked_scores, dim=-1)

        return attention_weights @ V

class MultiHeadAttention(nn.Module):
    """
    멀티 헤드 어텐션 메커니즘
    모델이 다양한 위치의 정보에 동시에 주목할 수 있도록 합니다
    """
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        d_h = emb_dim // num_heads  # 각 어텐션 헤드의 차원

        # 여러 어텐션 헤드 생성
        self.heads = nn.ModuleList([
            AttentionHead(emb_dim, d_h)
            for _ in range(num_heads)
        ])

        # 출력 투영 행렬
        self.W_O = nn.Parameter(torch.rand(emb_dim, emb_dim))

    def forward(self, x, mask):
        # 각 어텐션 헤드를 통해 입력 처리
        head_outputs = [head(x, mask) for head in self.heads]

        # 출력을 연결하고 최종 차원으로 투영
        x = torch.cat(head_outputs, dim=-1)
        return x @ self.W_O

class MLP(nn.Module):
    """
    트랜스포머 피드-포워드 네트워크를 위한 다층 퍼셉트론
    ReLU 활성화를 사용하여 더 큰 중간 차원(4배)을 사용합니다
    """
    def __init__(self, emb_dim):
        super().__init__()
        # 2층 피드-포워드 네트워크에 대한 가중치와 편향 초기화
        self.W_1 = nn.Parameter(torch.rand(emb_dim, emb_dim * 4))
        self.B_1 = nn.Parameter(torch.rand(emb_dim * 4))
        self.W_2 = nn.Parameter(torch.rand(emb_dim * 4, emb_dim))
        self.B_2 = nn.Parameter(torch.rand(emb_dim))

    def forward(self, x):
        # 첫 번째 선형 변환과 활성화
        x = x @ self.W_1 + self.B_1
        x = torch.relu(x)
        # 두 번째 선형 변환
        x = x @ self.W_2 + self.B_2
        return x

class DecoderBlock(nn.Module):
    """
    단일 트랜스포머 디코더 블록
    잔차 연결이 있는 셀프 어텐션과 피드-포워드 층을 결합합니다
    """
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        # 층 구성 요소
        self.norm1 = RMSNorm(emb_dim)
        self.attn = MultiHeadAttention(emb_dim, num_heads)
        self.norm2 = RMSNorm(emb_dim)
        self.mlp = MLP(emb_dim)

    def forward(self, x, mask):
        # 잔차 연결이 있는 셀프 어텐션 하위 블록
        attn_out = self.attn(self.norm1(x), mask)
        x = x + attn_out

        # 잔차 연결이 있는 피드-포워드 하위 블록
        mlp_out = self.mlp(self.norm2(x))
        x = x + mlp_out

        return x

class DecoderLanguageModel(nn.Module):
    """
    완전한 디코더 전용 트랜스포머 언어 모델
    여러 디코더 블록을 사용하여 입력 시퀀스를 처리하고 어휘로 투영합니다
    """
    def __init__(self, vocab_size, emb_dim, num_heads, num_blocks, pad_idx):
        super().__init__()
        # 토큰 임베딩 층
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)

        # 디코더 블록 스택
        self.layers = nn.ModuleList([
            DecoderBlock(emb_dim, num_heads) for _ in range(num_blocks)
        ])

        # 어휘 크기로의 출력 투영
        self.output = nn.Parameter(torch.rand(emb_dim, vocab_size))

    def forward(self, x):
        # 입력 토큰 임베딩
        x = self.embedding(x)

        # 코잘 어텐션 마스크 생성
        _, seq_len, _ = x.size()
        mask = torch.tril(torch.ones(seq_len, seq_len, device=x.device))

        # 디코더 블록을 통해 처리
        for layer in self.layers:
            x = layer(x, mask)

        # 어휘 분포로 투영
        return x @ self.output

## 언어 모델 훈련

다음 셀에서 데이터를 로드하고, 모델을 훈련 및 저장합니다.

In [ ]:
# ----------------------------
# 디코더 언어 모델을 위한 메인 훈련 루프
# 이 스크립트는 데이터 로딩, 모델 훈련, 검증 및 텍스트 생성을 포함한
# 전체 훈련 과정을 처리합니다.
# ----------------------------
if __name__ == "__main__":
    # 재현 가능한 결과를 보장하기 위해 난수 시드 초기화
    set_seed(42)

    # 구성에서 모델 아키텍처 및 훈련 하이퍼파라미터 검색
    # emb_dim: 입력 토큰 및 중간 임베딩의 차원
    # num_heads: 각 트랜스포머 블록의 어텐션 헤드 수
    # num_blocks: 모델의 트랜스포머 블록 수
    # batch_size: 미니 배치 크기
    # learning_rate: 옵티마이저 업데이트를 위한 단계 크기
    # num_epochs: 훈련 데이터셋을 완전히 통과하는 횟수
    # context_size: 최대 입력 시퀀스 길이
    emb_dim, num_heads, num_blocks, batch_size, learning_rate, num_epochs, context_size = get_hyperparameters()

    # 마이크로소프트의 Phi-3.5-mini 모델을 사용하여 토크나이저 초기화
    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

    # 더 짧은 시퀀스에 대한 패딩을 위한 패딩 토큰 인덱스 가져오기
    pad_idx = tokenizer.pad_token_id

    # CUDA 지원 GPU가 있는지 확인하고 그에 따라 장치를 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 뉴스 데이터셋 다운로드 및 훈련과 테스트를 위한 DataLoader 객체 생성
    # DataLoader는 배칭 및 셔플링을 처리합니다
    data_url = "https://www.thelmbook.com/data/news"
    train_dataloader, test_dataloader = download_and_prepare_data(
        data_url, batch_size, tokenizer, context_size
    )

    # 모델이 처리해야 할 어휘의 크기 가져오기
    vocab_size = len(tokenizer)
    print(f"\n어휘 크기: {vocab_size}\n")

    # 지정된 아키텍처 매개변수로 디코더 언어 모델 초기화
    # vocab_size: 출력 층 차원 결정
    # emb_dim: 토큰 임베딩 및 중간 임베딩의 크기
    # num_heads: 트랜스포머 블록당 어텐션 헤드 수
    # num_blocks: 모델의 트랜스포머 블록 수
    # pad_idx: 더 짧은 시퀀스에 대한 패딩에 사용되는 특수 토큰 ID
    model = DecoderLanguageModel(
        vocab_size, emb_dim, num_heads, num_blocks, pad_idx
    )

    # 사용 가능한 경우 모델을 GPU로 이동
    model.to(device)

    # 사용자 지정 초기화 방식을 사용하여 모델 가중치 초기화
    # 이는 심층 신경망의 안정적인 훈련에 중요합니다
    initialize_weights(model)

    # 지정된 학습률로 AdamW 옵티마이저 초기화
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # 훈련을 위한 손실 함수(교차 엔트로피) 초기화
    # ignore_index=pad_idx는 패딩 토큰이 손실에 기여하지 않도록 보장합니다
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

    # 모델의 총 훈련 가능한 매개변수 수 계산 및 표시
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\n총 훈련 가능한 매개변수: {total_params}\n")

    # 평가 간격 설정(검증을 수행할 샘플 수)
    # 200,000 샘플은 훈련 시간과 모니터링 빈도 사이의 좋은 균형을 제공합니다
    eval_interval = 200_000
    examples_processed = 0  # 다음 평가까지의 진행 상황을 추적하기 위한 카운터

    # 평가 중에 샘플 텍스트를 생성하기 위한 테스트 문맥 정의
    contexts = [
        "Moscow",
        "New York",
        "A hurricane",
        "The President"
    ]

    # 메인 훈련 루프 - 지정된 수의 에포크 반복
    for epoch in range(num_epochs):
        # 모델을 훈련 모드로 설정
        model.train()

        # 이 에포크를 위한 추적 변수 초기화
        total_loss = 0.0      # 모든 배치에 대한 손실 누적 변수
        total_tokens = 0      # 처리된 실제 토큰 수 카운터 (패딩 제외)

        # 훈련 진행 상황 모니터링을 위한 진행률 표시줄 생성
        progress_bar = tqdm(train_dataloader, desc=f"에포크 {epoch+1}/{num_epochs}")

        # 훈련 데이터의 배치 반복
        for batch_idx, (input_seq, target_seq) in enumerate(progress_bar):
            # 사용 가능한 경우 입력 및 타깃 시퀀스를 GPU로 이동
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            # 이전 배치의 그레이디언트 지우기
            optimizer.zero_grad()

            # 포워드 패스: 이 배치에 대한 모델 예측 가져오기
            # 출력 크기: (batch_size, seq_len, vocab_size)
            logits = model(input_seq)

            # 손실 계산을 위해 로짓 및 타깃 텐서 재구성
            logits = logits.reshape(-1, logits.size(-1))
            target = target_seq.reshape(-1)

            # 손실 계산에서 패딩 토큰을 제외하는 마스크 생성
            mask = target != pad_idx

            # 모델 예측과 실제 타깃 간의 손실 계산
            # 마스크된 버전을 사용하여 패딩 토큰 무시
            loss = criterion(logits[mask], target[mask])

            # 백워드 패스: 모델 매개변수에 대한 손실의 그레이디언트 계산
            loss.backward()

            # 계산된 그레이디언트를 사용하여 모델 매개변수 업데이트
            optimizer.step()

            # 패딩을 고려한 이 배치에 대한 실제 손실 값 계산
            loss_value = loss.item() * mask.sum().item()

            # 에포크 통계를 위한 총 손실 및 토큰 누적
            total_loss += loss_value
            total_tokens += mask.sum().item()
            examples_processed += input_seq.size(0)

            # 현재 배치 손실로 진행률 표시줄 업데이트
            progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

            # 지정된 수의 샘플을 처리한 후 주기적 평가
            if examples_processed >= eval_interval:
                # 마지막 eval_interval 샘플에 대한 평균 손실 계산
                avg_loss = total_loss / total_tokens
                print(f"\n{examples_processed} 샘플 처리 후, 평균 손실: {avg_loss:.4f}")

                # 평가 모드로 전환
                model.eval()

                # 검증 메트릭 계산
                average_loss, perplexity = compute_loss_and_perplexity(
                    model, test_dataloader, tokenizer, criterion, device, max_sentences=1000
                )

                # 검증 기록
                print(f"\n검증 평균 손실: {average_loss:.4f}, 혼잡도: {perplexity:.2f}")

                model.eval()

                # 모델 성능을 정성적으로 평가하기 위해 샘플 텍스트 생성
                for context in contexts:
                    # 각 테스트 문맥에 대한 텍스트 연속 생성
                    generated_text = generate_text(
                        model=model,
                        start_string=context,
                        tokenizer=tokenizer,
                        device=device,
                        max_length=50
                    )
                    print(f"\n문맥: {context}")
                    print(f"\n생성된 텍스트: {generated_text}\n")

                # 계속 훈련하기 위해 훈련 모드로 다시 전환
                model.train()

                # 다음 평가 간격을 위해 카운터 재설정
                examples_processed = 0
                total_loss = 0.0
                total_tokens = 0

        # 에포크 종료 보고
        if total_tokens > 0:
            # 에포크에 대한 평균 손실 계산 및 표시
            avg_loss = total_loss / total_tokens
            print(f"\n에포크 {epoch+1}/{num_epochs}, 평균 손실: {avg_loss:.4f}")
        else:
            # 처리된 토큰이 없는 경우의 엣지 케이스 처리
            print(f"\n에포크 {epoch+1}/{num_epochs} 완료.")

        # 에포크 종료 검증 수행
        model.eval()

        # 정성적 평가를 위한 샘플 텍스트 생성
        print("\ngenerate_text를 사용하여 문맥을 기반으로 텍스트 생성:\n")
        for context in contexts:
            generated_text = generate_text(
                model=model,
                start_string=context,
                tokenizer=tokenizer,
                device=device,
                max_length=50
            )
            print(f"\n문맥: {context}")
            print(f"\n생성된 텍스트: {generated_text}\n")

        average_loss, perplexity = compute_loss_and_perplexity(
            model, test_dataloader, tokenizer, criterion, device, max_sentences=1000
        )
        print(f"\n검증 평균 손실: {average_loss:.4f}, 혼잡도: {perplexity:.2f}")

        # 다음 에포크를 위해 훈련 모드로 재설정
        model.train()

    # 나중에 사용하기 위해 훈련된 모델과 토크나이저 저장
    # 여기에는 모델 아키텍처, 가중치 및 토크나이저 구성이 포함됩니다
    model_name = "Decoder_LM"
    save_model(model, tokenizer, model_name)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

Epoch 1/1: 100%|█████████▉| 171932/172148 [3:32:33<01:44,  2.07it/s, loss=4.1245]


Context: The President

Generated text: The President 's office said the administration 's decision to address the issue of the president 's actions . '' 'Yes . ' '' ' I . ' '' 's . 's . 's . 's . 's . ' .



Epoch 1/1: 100%|██████████| 172148/172148 [3:32:48<00:00, 13.48it/s, loss=4.0147]



Epoch 1/1, Average Loss: 4.0339

Generating text based on contexts using generate_text:


Context: Moscow

Generated text: Moscow , which has been the first of its ##,###-strong .###-###-###-### .### .### . 's . 's the . 'ed . ' '' . 's the


Context: New York

Generated text: New York City Mayor Michael Bloomberg said the company 's decision to take a breakdown of the new deal . '' 's . ' '' ' I . 's . 's . ' '' . 's . ' . ' . '


Context: A hurricane

Generated text: A hurricane season is expected to be completed in #### . '' ' I 'm not going to be a major factor . ' '' ' I 'm not going to be a matter of . ' '' ' . ' '' ' . ' ''


Context: The President

Generated text: The President of the House of Representatives has said that the bill would be a `` very important step '' . '' ' '' . ' '' ' I 'm not sure that the president of the House of Representatives of the House of Representatives . '' `` the




Validation Average Loss: 4.0108, Perplexity: 55.19
Model and tokenizer saved as 'Decoder_LM'


## 모델 테스트하기

아래 셀에서 언어 모델을 로드하여 텍스트를 생성합니다.

In [ ]:
# ----------------------------
# Model tests
# ----------------------------

if __name__ == "__main__":

    model_name = "Decoder_LM"

    # Load the previously saved model and tokenizer from disk
    # This recreates the exact model state from after training
    model, tokenizer = load_model(model_name)

    model.eval()

    # Print header for test section
    print("\nTesting the model:\n")

    # Define a list of test prompts to evaluate model performance
    contexts = [
        "Moscow",
        "New York",
        "A hurricane",
        "The President"
    ]

    # Iterate through each test prompt and generate text
    for context in contexts:
        # Generate text using greedy decoding (most likely tokens)
        generated_text = generate_text(
            model=model,          # The loaded language model
            start_string=context, # Text to continue
            tokenizer=tokenizer,  # Tokenizer for text conversion
            device=device,        # CPU or GPU device
            max_length=50         # Maximum length of generated sequence
        )
        # Print the original prompt and model's response
        print(f"\nPrompt: {context}")
        print(f"\nGenerated response: {generated_text}\n")


Model 'Decoder_LM' loaded successfully

Testing the model:


Prompt: Moscow

Generated response: Moscow , which has been the first of its ##,###-strong .###-###-###-### .### .### . 's . 's the . 'ed . ' '' . 's the


Prompt: New York

Generated response: New York City Mayor Michael Bloomberg said the company 's decision to take a breakdown of the new deal . '' 's . ' '' ' I . 's . 's . ' '' . 's . ' . ' . '


Prompt: A hurricane

Generated response: A hurricane season is expected to be completed in #### . '' ' I 'm not going to be a major factor . ' '' ' I 'm not going to be a matter of . ' '' ' . ' '' ' . ' ''


Prompt: The President

Generated response: The President of the House of Representatives has said that the bill would be a `` very important step '' . '' ' '' . ' '' ' I 'm not sure that the president of the House of Representatives of the House of Representatives . '' `` the

